In [31]:
import QUANTAXIS as QA
import pandas as pd
import numpy as np
from pprint import  pprint as print
import sklearn as sk
from sklearn import preprocessing

def func_scale(df,groups):
    #部分不同列的数据具有相关性，需要统一标准化
    for group in groups:
        if type(group)!=list and type(group)!=tuple:
            group=[group]
        assemble_scale_column=group
        df[assemble_scale_column] = preprocessing.scale(np.ravel(df[assemble_scale_column])).reshape([len(df),-1])
    return df
def standardization(df,groups):
    #对数据去中心化，使符合正态分布
    df=df.groupby('code').apply(func_scale,groups)
    return df

def func_series_to_supervised(dataset, n_in=1):
    #dataset = pd.DataFrame(data)
    df_all=pd.DataFrame()
    
    for i in range(n_in, 0, -1):
        df_back=dataset.shift(i)
        col_names=df_back.columns+'_'+str(i)
        df_back.columns=col_names
        df_all=pd.concat([df_all,df_back],axis=1)
        #print(df_all.index)
    return df_all
#series_to_supervised(dataset,3)
def series_to_supervised(dataset, n_in=1):
    return dataset.groupby('code').apply(func_series_to_supervised,n_in)

def max_min_close_n_days(data, n=30):
    '''
    自定义指标，计算当日（不含）到n天后收盘价最大值、最小值、最大值上涨比率、最小值下跌比率
    '''
    n_days_max=pd.Series(np.zeros(len(data)))
    n_days_min=pd.Series(np.zeros(len(data)))
    n_days_max_radio=pd.Series(np.zeros(len(data)))
    n_days_min_radio=pd.Series(np.zeros(len(data)))
    if(len(data)-n<0):
        '''for i in range(len(data)):
            n_days_max[i]=None
            n_days_min[i]=None'''
        n_days_max[0:len(data)]=None
        n_days_min[0:len(data)]=None
    else:
        for i in range(len(data)-n):
            n_days_max[i]=(max(data['close'][i+1:i+n+1]))
            n_days_min[i]=(min(data['close'][i+1:i+n+1]))
            n_days_max_radio[i]=(n_days_max[i]-data['close'][i])/data['close'][i]
            n_days_min_radio[i]=(n_days_min[i]-data['close'][i])/data['close'][i]
        n_days_max[len(data)-n:len(data)]=None
        n_days_min[len(data)-n:len(data)]=None
        '''for i in range(len(data)-n,len(data)):
            n_days_max[i]=None
            n_days_min[i]=None
            '''
    max_min_close_n_days=pd.DataFrame({'n_days_max':n_days_max,'n_days_min':n_days_min,
                                       'n_days_max_radio':n_days_max_radio,'n_days_min_radio':n_days_min_radio})
    max_min_close_n_days.index=data.index
    return max_min_close_n_days

def get_all_indicator(data):
    dataset=pd.DataFrame()

    indicator=QA.QA_indicator_ADTM(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_ARBR(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_ASI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_ATR(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_BBI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    #indicator=QA.QA_indicator_BIAS(data)
    #dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_BOLL(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_CCI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_CHO(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_DDI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_DMA(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_DMI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    #indicator=QA.QA_indicator_EMA(data)
    #dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_EXPMA(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_KDJ(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MA(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MA_VOL(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MACD(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MFI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MIKE(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_MTM(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_OBV(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_OSC(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_PBX(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_PVT(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_ROC(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_RSI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_VPT(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_VR(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_VRSI(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    indicator=QA.QA_indicator_VSTD(data)
    dataset=pd.concat([dataset,indicator],axis=1)

    #indicator=QA.QA_indicator_WR(data)
    #dataset=pd.concat([dataset,indicator],axis=1)
    
    return dataset


In [ ]:
k=100
n_days=1
stockCode=QA.QA_fetch_stock_list()['code'].tolist()
stockCode=stockCode[0:k]
data = QA.QA_fetch_stock_day_adv(stockCode, '2019-01-01', '2019-09-30')

#1、生成标签（y）
#获取当前股票20天后的最大涨幅和最小涨幅
Y = data.add_func(max_min_close_n_days,n_days)
#dataset=pd.concat([y,data.data], axis=1).dropna(thresh=9)

#2、生成样本（x）
dataset1=get_all_indicator(data)

#3、标准化样本数据
dataset2=standardization(data.data,(['open','close','high','low'],'volume','amount'))

#####4、合并样本
dataset=pd.concat([dataset1,dataset2],axis=1)

#####5、时间序列转监督学习（n天的数据生成一个样本）
dataset=series_to_supervised(dataset,10)

#####定义Y值，加入样本中
Y_names=['n_days_max_radio','n_days_min_radio']
dataset=pd.concat([dataset,Y[Y_names]],axis=1)

#####删除空值超过90%的样本
len_columns=len(dataset.columns)
dataset=dataset.dropna(thresh=len_columns*0.9)

#####生成X,Y
dataset=dataset.astype('float16')
Y=dataset[Y_names]
X=dataset.drop(Y_names,axis=1)

X_train,Y_train=X,Y




stockCode=QA.QA_fetch_stock_list()['code'].tolist()
stockCode=stockCode[0:k]
data = QA.QA_fetch_stock_day_adv(stockCode, '2019-10-01', '2019-12-31')

#1、生成标签（y）
#获取当前股票20天后的最大涨幅和最小涨幅
Y = data.add_func(max_min_close_n_days,n_days)
#dataset=pd.concat([y,data.data], axis=1).dropna(thresh=9)

#2、生成样本（x）
dataset1=get_all_indicator(data)

#3、标准化样本数据
dataset2=standardization(data.data,(['open','close','high','low'],'volume','amount'))

#####4、合并样本
dataset=pd.concat([dataset1,dataset2],axis=1)

#####5、时间序列转监督学习（n天的数据生成一个样本）
dataset=series_to_supervised(dataset,10)

#####定义Y值，加入样本中
Y_names=['n_days_max_radio','n_days_min_radio']
dataset=pd.concat([dataset,Y[Y_names]],axis=1)

#####删除空值超过90%的样本
len_columns=len(dataset.columns)
dataset=dataset.dropna(thresh=len_columns*0.9)

#####生成X,Y
dataset=dataset.astype('float16')
Y=dataset[Y_names]
X=dataset.drop(Y_names,axis=1)

X_test,Y_test=X,Y

In [47]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
#X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.3,random_state=7)
Y_train=Y_train['n_days_max_radio']
Y_test=Y_test['n_days_max_radio']

In [53]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l1'},
    'num_leaves': 31,
    'learning_rate': 0.001,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.4,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads':40
}

lgb_train = lgb.Dataset(X_train,Y_train)
lgb_eval = [lgb.Dataset(X_test,Y_test,reference=lgb_train),lgb_train]
model=lgb.train(params,lgb_train,num_boost_round=3000,valid_sets=lgb_eval,early_stopping_rounds=400)
    

[1]	training's l1: 0.0191759	valid_0's l1: 0.0132499
Training until validation scores don't improve for 400 rounds
[2]	training's l1: 0.0191732	valid_0's l1: 0.0132494
[3]	training's l1: 0.0191703	valid_0's l1: 0.013249
[4]	training's l1: 0.0191675	valid_0's l1: 0.0132485
[5]	training's l1: 0.0191646	valid_0's l1: 0.013248
[6]	training's l1: 0.0191616	valid_0's l1: 0.0132473
[7]	training's l1: 0.0191586	valid_0's l1: 0.0132468
[8]	training's l1: 0.0191552	valid_0's l1: 0.0132467
[9]	training's l1: 0.019152	valid_0's l1: 0.0132465
[10]	training's l1: 0.0191488	valid_0's l1: 0.013246
[11]	training's l1: 0.0191458	valid_0's l1: 0.0132457
[12]	training's l1: 0.019143	valid_0's l1: 0.0132447
[13]	training's l1: 0.01914	valid_0's l1: 0.0132442
[14]	training's l1: 0.0191373	valid_0's l1: 0.0132437
[15]	training's l1: 0.0191345	valid_0's l1: 0.0132433
[16]	training's l1: 0.0191316	valid_0's l1: 0.0132437
[17]	training's l1: 0.0191291	valid_0's l1: 0.0132433
[18]	training's l1: 0.0191261	valid_

In [67]:
b=dataset.loc[(slice(None),'000002'),:]
vvv=b['volume']
print(vvv.mean())
print(vvv.std())

m=700696-vvv.mean()
print(m)
s=m/vvv.std()
print(s)


353008.97959183675
145956.15940147487
347687.02040816325
2.382133250381004


In [16]:
stockCode=QA.QA_fetch_stock_list()['code'].tolist()
data = QA.QA_fetch_stock_day_adv(stockCode, '2018-03-20', '2018-08-23')
#print(stockCode)

In [5]:
# data.data
# Account=QA.QA_Account()
# Broker=QA.QA_BacktestBroker()
user = QA.QA_user(username ='quantaxis', password = 'quantaxis')
portfotlio=user.new_portfolio('x1')
account = portfolio.new_account(args)

TypeError: __init__() takes at least 2 positional arguments (0 given)

In [7]:
QA.__version__

'1.8.0'